## BASE SENSITIVITY ANALYSIS

In [109]:
import dolfin as dolf
from math import pi, sqrt
import matplotlib.pyplot as plt
import numpy as np

from helmholtz_pkg.passive_flame import PassiveFlame
from helmholtz_pkg.flame_transfer_function import n_tau
from helmholtz_pkg.active_flame import ActiveFlame
from helmholtz_pkg.eigensolvers import fixed_point_iteration_pep
from helmholtz_pkg.eigenvectors import normalize_eigenvector
from helmholtz_pkg.petsc4py_utils import vector_matrix_vector, vector_vector
import params

In [110]:
def mshr(el):

    mesh = dolf.UnitIntervalMesh(el)

    def l_boundary_func(x, on_boundary):
        x = x[0]
        return on_boundary and dolf.near(x, 0.)

    def r_boundary_func(x, on_boundary):
        x = x[0]
        return on_boundary and dolf.near(x, 1.)

    boundaries = dolf.MeshFunction('size_t', mesh, mesh.topology().dim() - 1)

    l_boundary = dolf.AutoSubDomain(l_boundary_func)
    r_boundary = dolf.AutoSubDomain(r_boundary_func)

    l_boundary.mark(boundaries, 1)
    r_boundary.mark(boundaries, 2)

    # ________________________________________________________________________________

    def fl_subdomain_func(x):
        x = x[0]
        x_f = params.x_f[0][0]
        a_f = params.a_f
        return x_f - a_f - dolf.DOLFIN_EPS <= x <= x_f + a_f + dolf.DOLFIN_EPS

    subdomains = dolf.MeshFunction('size_t', mesh, mesh.topology().dim())

    subdomains.set_all(1)

    fl_subdomain = dolf.AutoSubDomain(fl_subdomain_func)
    fl_subdomain.mark(subdomains, 0)

    return mesh, boundaries, subdomains

In [111]:
degree = 1

mesh, boundaries, subdomains = mshr(400)

boundary_conditions = {1: {'Robin': params.Y_in},  # inlet
                        2: {'Robin': params.Y_out}}  # outlet

foo = PassiveFlame(mesh, boundaries, boundary_conditions,
                    c=params.c,
                    degree=degree)
foo.assemble_A()
foo.assemble_B()
foo.assemble_C()

ftf = n_tau(params.n, params.tau)

D = ActiveFlame(mesh, subdomains,
                params.x_f, params.x_r, params.rho_in, 1., 1., ftf,
                degree=degree)
                
D.assemble_submatrices(problem_type='direct')
D.assemble_submatrices(problem_type='adjoint')

E_dir = fixed_point_iteration_pep(foo, D, pi, nev=2, i=0, problem_type='direct')
E_adj = fixed_point_iteration_pep(foo, D, pi, nev=2, i=1, problem_type='adjoint')

omega_dir, p_dir = normalize_eigenvector(mesh, E_dir, i=0, degree=degree, which='right')
omega_adj, p_adj = normalize_eigenvector(mesh, E_adj, i=1, degree=degree, which='left')

iter =  1,  omega = +3.42658955  +0.01448821j,  |domega| = 1.28e-02
iter =  2,  omega = +3.42440924  +0.00224480j,  |domega| = 1.24e-02
iter =  3,  omega = +3.42440954  +0.00224595j,  |domega| = 1.19e-06
iter =  4,  omega = +3.42440954  +0.00224595j,  |domega| = 5.62e-11
iter =  1,  omega = +3.42658955  -0.01448821j,  |domega| = 1.28e-02
iter =  2,  omega = +3.42440924  -0.00224480j,  |domega| = 1.24e-02
iter =  3,  omega = +3.42440954  -0.00224595j,  |domega| = 1.19e-06
iter =  4,  omega = +3.42440954  -0.00224595j,  |domega| = 5.62e-11
Normalization relation is | p p dx = 1 
Normalization relation is | p p dx = 1 


In [112]:
# BASE STATE SENSITIVITY ------------------------------------------------------------------------------------------

p_dir_vec = p_dir.vector().vec()
p_adj_vec = p_adj.vector().vec()

dLds = -D.get_derivative(omega_dir) + foo.B + foo.assemble_zC(2 * omega_dir)

denominator = -vector_matrix_vector(p_adj_vec , dLds , p_dir_vec )
#print(denominator)


D.assemble_matrix(omega_dir)
# sensitivity for n
dNdn = D.matrix / params.n
nom_n = -vector_matrix_vector(p_adj_vec , dNdn , p_dir_vec )
dsdn = nom_n / denominator
print("Sensitivity of ds/dn: ",dsdn)

# sensitivity for tau
D.assemble_matrix(1j*omega_dir)
dNdtau = D.matrix 
nom_tau = -vector_matrix_vector(p_adj_vec , dNdtau , p_dir_vec )
dsdtau = nom_tau / denominator
print("Sensitivity of ds/dtau: ",dsdtau)

# sensitivity for v
dsdv = np.zeros(400, dtype=complex)

D.assemble_matrix(omega_dir)
for i,cell in enumerate(dolf.cells(mesh)):

    # local_m = dolf.assemble_local(params.rho*dx, cell)
    local_m = (-foo.A*params.rho_in-D.matrix/params.rho_in)
    # nom_v = local_m * vector_vector(p_adj_vec, p_dir_vec)
    nom_v = -vector_matrix_vector(p_adj_vec , local_m , p_dir_vec )
    dsdv[i] =  nom_v/denominator
    #print(dsdv[i])

# plt.plot(abs(dsdv))
# plt.show()
D.matrix.size

Sensitivity of ds/dn:  (-0.023077920308460266+0.18023864616011248j)
Sensitivity of ds/dtau:  (0.005148156290955588-0.00022173542676997684j)


(802, 802)

In [113]:
W = foo.function_space

# MATRIX dA
(u_1, u_2) = foo.u
(v_1, v_2) = foo.v

da_11 = - params.gamma * params.p_amb * dolf.dot(dolf.grad(v_1), dolf.grad(u_1)) #* dx
da_22 = - params.gamma * params.p_amb * dolf.dot(dolf.grad(v_2), dolf.grad(u_2)) #* dx
da_ = da_11 + da_22

# # MATRIX dD
# (u_1, u_2) = foo.u
# (v_1, v_2) = foo.v



# # dx = dolf.Measure('dx', subdomain_data=self.subdomains)

# V = dolf.FunctionSpace(mesh, 'CG', 1)
# const = dolf.interpolate(dolf.Constant(1), V)
# V_fl = dolf.assemble(const * dx(fl))

# dd_1 = D.coeff * params.rho_in * dolf.assemble(v_1 / V_fl) #* dx(fl))
# dd_2 = D.coeff * params.rho_in * dolf.assemble(v_2  / V_fl) #* dx(fl))



In [114]:
V = dolf.FunctionSpace(mesh,"CG",1)
df = V.dofmap()
print(df)


In [115]:
dofmap = W.dofmap()
element = W.element()
for i, cell in enumerate(dolf.cells(mesh)):
    print(cell.index(), dofmap.cell_dofs(cell.index()))
    print("Cell Coordinates: \n", element.tabulate_dof_coordinates(cell))

0 [800 798 801 799]
Cell Coordinates: 
 [[0.    ]
 [0.0025]
 [0.    ]
 [0.0025]]
1 [798 796 799 797]
Cell Coordinates: 
 [[0.0025]
 [0.005 ]
 [0.0025]
 [0.005 ]]
2 [796 794 797 795]
Cell Coordinates: 
 [[0.005 ]
 [0.0075]
 [0.005 ]
 [0.0075]]
3 [794 792 795 793]
Cell Coordinates: 
 [[0.0075]
 [0.01  ]
 [0.0075]
 [0.01  ]]
4 [792 790 793 791]
Cell Coordinates: 
 [[0.01  ]
 [0.0125]
 [0.01  ]
 [0.0125]]
5 [790 788 791 789]
Cell Coordinates: 
 [[0.0125]
 [0.015 ]
 [0.0125]
 [0.015 ]]
6 [788 786 789 787]
Cell Coordinates: 
 [[0.015 ]
 [0.0175]
 [0.015 ]
 [0.0175]]
7 [786 784 787 785]
Cell Coordinates: 
 [[0.0175]
 [0.02  ]
 [0.0175]
 [0.02  ]]
8 [784 782 785 783]
Cell Coordinates: 
 [[0.02  ]
 [0.0225]
 [0.02  ]
 [0.0225]]
9 [782 780 783 781]
Cell Coordinates: 
 [[0.0225]
 [0.025 ]
 [0.0225]
 [0.025 ]]
10 [780 778 781 779]
Cell Coordinates: 
 [[0.025 ]
 [0.0275]
 [0.025 ]
 [0.0275]]
11 [778 776 779 777]
Cell Coordinates: 
 [[0.0275]
 [0.03  ]
 [0.0275]
 [0.03  ]]
12 [776 774 777 775]
Cell 

237 [326 324 327 325]
Cell Coordinates: 
 [[0.5925]
 [0.595 ]
 [0.5925]
 [0.595 ]]
238 [324 322 325 323]
Cell Coordinates: 
 [[0.595 ]
 [0.5975]
 [0.595 ]
 [0.5975]]
239 [322 320 323 321]
Cell Coordinates: 
 [[0.5975]
 [0.6   ]
 [0.5975]
 [0.6   ]]
240 [320 318 321 319]
Cell Coordinates: 
 [[0.6   ]
 [0.6025]
 [0.6   ]
 [0.6025]]
241 [318 316 319 317]
Cell Coordinates: 
 [[0.6025]
 [0.605 ]
 [0.6025]
 [0.605 ]]
242 [316 314 317 315]
Cell Coordinates: 
 [[0.605 ]
 [0.6075]
 [0.605 ]
 [0.6075]]
243 [314 312 315 313]
Cell Coordinates: 
 [[0.6075]
 [0.61  ]
 [0.6075]
 [0.61  ]]
244 [312 310 313 311]
Cell Coordinates: 
 [[0.61  ]
 [0.6125]
 [0.61  ]
 [0.6125]]
245 [310 308 311 309]
Cell Coordinates: 
 [[0.6125]
 [0.615 ]
 [0.6125]
 [0.615 ]]
246 [308 306 309 307]
Cell Coordinates: 
 [[0.615 ]
 [0.6175]
 [0.615 ]
 [0.6175]]
247 [306 304 307 305]
Cell Coordinates: 
 [[0.6175]
 [0.62  ]
 [0.6175]
 [0.62  ]]
248 [304 302 305 303]
Cell Coordinates: 
 [[0.62  ]
 [0.6225]
 [0.62  ]
 [0.6225]]
249 

In [122]:
p_dir_r, p_dir_i = p_dir.split(True)
p_adj_r, p_adj_i = p_adj.split(True)

p_dir_array = p_dir_r.compute_vertex_values() + 1j*p_dir_i.compute_vertex_values()
p_adj_array = p_adj_r.compute_vertex_values() + 1j*p_adj_i.compute_vertex_values()

dsdparam = np.zeros(400)
dx = dolf.Measure('dx', domain = mesh)

string_f_1d = '1 / (sigma * sqrt(2*pi)) * exp(- pow(x[0] - x_0, 2) / (2 * pow(sigma, 2)) )'

v = dolf.Expression(string_f_1d, degree=0, x_0=params.x_f[0][0], sigma=params.a_f)
func = dolf.interpolate(v, V)

for i, cell in enumerate(dolf.cells(mesh)):
    cell_id = cell.index()
    eig_dir = p_dir_array[cell_id]
    eig_adj = p_adj_array[cell_id]
    
    local = dolf.assemble_local(-da_*dx , cell)
    #local = dolf.assemble_local(func*dx, cell)
    print(local)
    
    nom = eig_dir * local * eig_adj
    denom = denominator
    dsdparam[cell_id] = nom/denom
    
plt.plot(abs(dsdparam))

[[ 56000000. -56000000.         0.         0.]
 [-56000000.  56000000.         0.         0.]
 [        0.         0.  56000000. -56000000.]
 [        0.         0. -56000000.  56000000.]]


ValueError: setting an array element with a sequence.

In [ ]:
dolf.plot(func)